In [1]:
from dotenv import load_dotenv

load_dotenv('./.env')

True

In [2]:
import os

print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")
os.environ['LANGCHAIN_PROJECT'] = 'RAG_test2'
print(f"[LANGCHAIN_PROJECT]\n{os.environ['LANGCHAIN_PROJECT']}")

[API KEY]
sk-l0rAOwEsqXqeXoZsj4jppfpH52TOFQezaNohE3ztURT3BlbkFJmqnZl5n_izzgDQoGAxMjXYm0nj4BrEzXWrArHJABUA
[LANGCHAIN_PROJECT]
RAG_test2


In [3]:
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("RAG_test2")

LangSmith 추적을 시작합니다.
[프로젝트명]
RAG_test2


In [4]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
### PDF 기반 QA(Question-Answering) 챗봇으로 변경하는 코드

from langchain.document_loaders import PyMuPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyMuPDFLoader("LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

문서의 수: 57

[페이지내용]


[metadata]
{'source': 'LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf', 'file_path': 'LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf', 'page': 10, 'total_pages': 57, 'format': 'PDF 1.7', 'title': '[대동그룹]DAY2_LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기', 'author': '양유정', 'subject': '', 'keywords': '', 'creator': 'Microsoft® PowerPoint® Microsoft 365용', 'producer': 'Microsoft® PowerPoint® Microsoft 365용', 'creationDate': "D:20241014091229+09'00'", 'modDate': "D:20241014091229+09'00'", 'trapped': ''}



/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 0 of document LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 1 of document LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 2 of document LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWarning: Warning: Empty content on page 3 of document LLM의 동작원리와 Langchain을 이용한 맞춤형 챗봇 만들기.pdf
  warnings.warn(
/home/elicer/.local/lib/python3.10/site-packages/langchain_community/document_loaders/parsers/pdf.py:299: UserWa

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents(docs)
len(splits)

12

In [7]:
# 벡터스토어를 생성합니다.
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# 뉴스에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [9]:
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)


# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
from langchain_teddynote.messages import stream_response

In [11]:
answer = rag_chain.stream("프롬프트는 무엇을 의미하나?")
# # 제너레이터에서 데이터를 하나씩 모아서 문자열로 합치기
# answer_content = ''.join([chunk for chunk in answer])

# # 결과 출력
# print(answer_content)
stream_response(answer)


주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다.